In [1]:
import sys
sys.path = ["../../.."] + sys.path # 切换到项目目录下

import scanpy as sc
import scvelo as scv
import velovgi

from ray import tune, air
from ray.air import session

Global seed set to 0
/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


## 1. 读取原始数据

In [2]:
adata = scv.read("../../../erythroid_lineage/data/erythroid_lineage.h5ad")

cluster_key = "celltype"
batch_key="stage"

adata

AnnData object with n_obs × n_vars = 500 × 53801
    obs: 'sample', 'stage', 'sequencing.batch', 'theiler', 'celltype'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'MURK_gene', 'Δm', 'scaled Δm'
    uns: 'celltype_colors'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced'

In [3]:
batch_pair_list = [
    ["E7.0", "E7.25"],
    ["E7.25", "E7.5"],
    ["E7.5", "E7.75"],
    ["E7.75", "E8.0"],
    ["E8.0", "E8.25"],
    ["E8.25", "E8.5"],
]

In [4]:
cluster_edges = [
    ("Blood progenitors 1", "Blood progenitors 2"), 
    ("Blood progenitors 2", "Erythroid1"), 
    ("Erythroid1", "Erythroid2"), 
    ("Erythroid2", "Erythroid3")
    ] # 已知的细胞类型间的分化信息

## 2. 在预处理部分调整构建邻居数量的两个参数

1. 目标函数

In [5]:
from pytorch_lightning import loggers
from torch_geometric import seed_everything

def train_velovgi(config):

    # 邻居数量的两个参数
    n_bnn_neighbors = config["n_bnn_neighbors"]
    n_knn_neighbors = config["n_knn_neighbors"]
    
    knn_mask, bnn_mask, subsample_adata = velovgi.pp.preprocess(adata, n_bnn_neighbors, n_knn_neighbors, sample_mode="random", batch_key=batch_key, batch_pair_list=batch_pair_list)

    name = ""
    for k,v in config.items():
        name += "%s_%s,"%(k, v)
    name = name[:-1]

    seed_everything(0)
    # 模型训练
    logger = loggers.TensorBoardLogger(save_dir="./log", name=name)
    velovgi.tl.VELOVGI.setup_anndata(adata=subsample_adata, spliced_layer="Ms", unspliced_layer="Mu")
    velovgi_model = velovgi.tl.VELOVGI(subsample_adata)
    velovgi_model.train(logger=logger)

    # 模型恢复
    velovgi.tl.add_velovi_outputs_to_adata(subsample_adata, velovgi_model) # 模型输出
    velovgi.pp.moment_recover(adata, subsample_adata) # 恢复

    # 速率计算
    scv.tl.velocity_graph(adata)
    scv.pl.velocity_embedding(adata, color=cluster_key)
    scv.pl.velocity_embedding_stream(adata, color=cluster_key, save="%s.png"%name)

    # 计算指标评价
    adata_velo = velovgi.tl.pre_metric(adata)
    exp_metrics = velovgi.tl.summary_metric(adata_velo, cluster_edges, cluster_key)[-1] # 计算指标汇总后的结果

    session.report({"CBDir": exp_metrics["CBDir"], "ICVCoh": exp_metrics["ICVCoh"]})


2. 搜索空间，网格搜索

In [6]:
search_space = {
    "n_bnn_neighbors": tune.grid_search([15, 45]),
    "n_knn_neighbors": tune.grid_search([15]),
}

3. 执行调参

In [7]:
from ray.tune.schedulers import ASHAScheduler

name = "test_experiment"

tuner = tune.Tuner(
    train_velovgi,
    tune_config=tune.TuneConfig(
        metric="CBDir",
        mode="max",
        scheduler=ASHAScheduler()
    ),
    run_config=air.RunConfig(
        local_dir="./results", # Trail内部具体输出结果在这里保存
        name=name # 开启调参的Tensorboard日志
    ),
    param_space=search_space,
)

results = tuner.fit()

2023-06-03 23:51:02,642	INFO worker.py:1625 -- Started a local Ray instance.
2023-06-03 23:51:04,261	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2023-06-03 23:51:04,510	INFO tensorboardx.py:172 -- pip install "ray[tune]" to see TensorBoard files.
2023-06-03 23:51:04,511	WARNING callback.py:142 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


2023-06-03 23:51:05,047	WARNING worker.py:1986 -- Warning: The actor ImplicitFunc is very large (26 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=2209) Global seed set to 0
(pid=2209) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(pid=2209)   new_rank_zero_deprecation(
(pid=2209) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_ex

(train_velovgi pid=2209) Filtered out 51490 genes that are detected 20 counts (shared).
(train_velovgi pid=2209) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=2209) Extracted 2000 highly variable genes.
(train_velovgi pid=2209) Logarithmized X.
(train_velovgi pid=2209) calculating knn and bnn mask...
(train_velovgi pid=2209) pair_list : [['E7.0', 'E7.25'], ['E7.25', 'E7.5'], ['E7.5', 'E7.75'], ['E7.75', 'E8.0'], ['E8.0', 'E8.25'], ['E8.25', 'E8.5']]


(pid=2272) Global seed set to 0
(pid=2272) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
(pid=2272)   new_rank_zero_deprecation(
(pid=2272) /usr/local/conda/envs/velovi-env/lib/python3.8/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
(pid=2272)   return new_rank_zero_deprecation(*args, **kwargs)


(train_velovgi pid=2272) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=2272) Normalized count data: X, spliced, unspliced.
(train_velovgi pid=2272) Extracted 2000 highly variable genes.
(train_velovgi pid=2272) Logarithmized X.
(train_velovgi pid=2272) calculating knn and bnn mask...
(train_velovgi pid=2272) pair_list : [['E7.0', 'E7.25'], ['E7.25', 'E7.5'], ['E7.5', 'E7.75'], ['E7.75', 'E8.0'], ['E8.0', 'E8.25'], ['E8.25', 'E8.5']]


...

(train_velovgi pid=2209)     finished (0:00:01) --> added 
(train_velovgi pid=2209)     'velocity_graph', sparse matrix with cosine correlations (adata.uns)
(train_velovgi pid=2209) computing velocity embedding
(train_velovgi pid=2209)     finished (0:00:00) --> added
(train_velovgi pid=2209)     'velocity_umap', embedded velocity vectors (adata.obsm)
(train_velovgi pid=2209) Figure(640x480)
(train_velovgi pid=2209) saving figure to file ./figures/scvelo_n_bnn_neighbors_15,n_knn_neighbors_15.png
(train_velovgi pid=2209) Figure(640x480)
(train_velovgi pid=2272) Figure(640x480)


Trial name,CBDir,ICVCoh,date,done,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
train_velovgi_72527_00000,0.738401,0.96508,2023-06-04_00-05-45,True,DESKTOP-9GVJMSD,1,172.24.199.116,2209,875.623,875.623,875.623,1685808345,1,72527_00000
train_velovgi_72527_00001,0.751919,0.956175,2023-06-04_00-06-10,False,DESKTOP-9GVJMSD,1,172.24.199.116,2272,894.051,894.051,894.051,1685808370,1,72527_00001


(train_velovgi pid=2272)   warn(
(train_velovgi pid=2272)   warn(


(train_velovgi pid=2272)   0%|          | 0/500 [00:00<?, ?cells/s]
(train_velovgi pid=2272)     finished (0:00:00) --> added 
(train_velovgi pid=2272)     'velocity_graph', sparse matrix with cosine correlations (adata.uns)
(train_velovgi pid=2272) computing velocity embedding
(train_velovgi pid=2272)     finished (0:00:00) --> added
(train_velovgi pid=2272)     'velocity_umap', embedded velocity vectors (adata.obsm)
(train_velovgi pid=2272) Figure(640x480)
(train_velovgi pid=2272) saving figure to file ./figures/scvelo_n_bnn_neighbors_45,n_knn_neighbors_15.png
(train_velovgi pid=2272) Figure(640x480)


2023-06-04 00:06:10,386	INFO tune.py:945 -- Total run time: 906.12 seconds (905.74 seconds for the tuning loop).


4. 查看训练日志与最优结果: tensorboard --logdir

In [8]:
results.get_best_result().config

{'n_bnn_neighbors': 45, 'n_knn_neighbors': 15}